In [15]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Import Data

In [96]:
ratings = pd.read_csv('rtg.csv')
movies = pd.read_csv('movies.csv')

In [97]:
movie_data = movies[['movieId', 'title']]
movie_data.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
llor_matrix = ratings[['userId', 'movieId', 'S']]

In [5]:
llor_matrix.head()

,userId,movieId,S
0,1,1,3.3
1,1,3,4.1
2,1,6,6.1
3,1,47,4.5
4,1,50,3.5


In [6]:
merge_book_ratings = pd.merge(llor_matrix, movie_data, on=['movieId'], how='inner')
merge_book_ratings.head()

,userId,movieId,S,title
0,1,1,3.300000,Toy Story (1995)
1,5,1,2.806738,Toy Story (1995)
2,7,1,5.000000,Toy Story (1995)
3,15,1,2.000000,Toy Story (1995)
4,17,1,5.850000,Toy Story (1995)


# Observasi Data

Menghitung banyaknya konten id yang paling sering di rating oleh pengguna

In [7]:
rating_count = pd.DataFrame(llor_matrix.groupby('movieId')['S'].count())
rating_count.sort_values('S', ascending=False).head()

,S
movieId,
356,329
318,317
296,307
593,279
2571,278


Menghitung rata2 dari nilai S dari data

In [8]:
average_rating = pd.DataFrame(llor_matrix.groupby('movieId')['S'].mean())
average_rating['ratingCount'] = pd.DataFrame(llor_matrix.groupby('movieId')['S'].count())
average_rating.sort_values('ratingCount', ascending=False).head()

,S,ratingCount
movieId,,
356,4.523208,329
318,4.717935,317
296,4.563681,307
593,4.466129,279
2571,4.377698,278


Ubah data menjadi matrix 2D untuk diproses menggunakan kNN

In [16]:
ratings_pivot = llor_matrix.pivot(index='movieId', columns='userId', values='S').fillna(0)
ratings_pivot_matrix = csr_matrix(ratings_pivot.values)

In [17]:
model = NearestNeighbors(metric='cosine', algorithm='brute', n_jobs=-1)
model.fit(ratings_pivot_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

# Testing Recommendation Model

In [102]:
idx = np.random.choice(ratings_pivot.shape[0])
distances, index = model.kneighbors(ratings_pivot.iloc[idx, :].values.reshape(1, -1), n_neighbors=5)

In [103]:
print('Create recommendation for :',idx)
print('List Index :',index)

Create recommendation for : 3229
List Index : [[3229 3910 2978 3281 5255]]


In [104]:
for i in range(len(distances.flatten())):
    if (i == 0):
        print('Recommendations for',movie_data[movie_data['movieId'] == idx].title.values[0])
    else:
        print('{0}: {1}, with distance : '.format(i, movie_data[movie_data['movieId'] == index.flatten()[i]].title.values[0]), distances.flatten()[i])

Recommendations for Another Man's Poison (1952)
1: Dancer in the Dark (2000), with distance :  0.4797904016605019
2: Three to Tango (1999), with distance :  0.49012445235751567
3: Brandon Teena Story, The (1998), with distance :  0.5079032973795996
4: Sorority Boys (2002), with distance :  0.5117884423301251
